<a href="https://colab.research.google.com/github/aletyska/crc_wsi_classification/blob/main/Trabalho_Grau_B_Alessandro_Tyska.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colorectal Cancer WSI Classification with CNN Neural Network
* UNISINOS - Universidade do Vale do Rio dos Sinos
* PPGCA - Programa de Pós-Graduação em Computação Aplicada
* Student: Alessandro Tyska
* Supervisor: Dr. Felipe Zeiser
* Course: Deep Learning

In [2]:
# @title Installing Packages
!pip install kagglehub

In [3]:
# @title Imports and Variables

import os
import kagglehub
import shutil

KAGGLE_DATASET_NAME = "mahdiislam/colorectal-cancer-wsi"
KAGGLE_DATASET_PATH = kagglehub.dataset_download(KAGGLE_DATASET_NAME)
KAGGLE_DATASET_PATH = os.path.join(KAGGLE_DATASET_PATH, "EBHI-SEG")

100%|██████████| 264M/264M [00:01<00:00, 143MB/s]

Extracting files...


In [8]:
# @title Basic Functions
def print_dataset_folder_structure(root_path):
    for root, dirs, files in os.walk(root_path):
        level = root.replace(root_path, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)

def restructure_dataset(root_path):
    if not os.path.exists(root_path):
        print(f"Path not found: {root_path}")
        return

    # Iterate over the class folders (High-grade IN, Normal, etc.)
    for class_name in os.listdir(root_path):
        class_dir = os.path.join(root_path, class_name)

        # Ensure we are processing directories only
        if os.path.isdir(class_dir):
            print(f"Processing: {class_name}")

            # 1. Remove 'label' folder
            label_dir = os.path.join(class_dir, 'label')
            if os.path.exists(label_dir):
                shutil.rmtree(label_dir)
                print(f"  - Removed label folder")

            # 2. Move files from 'image' folder to the class folder
            image_dir = os.path.join(class_dir, 'image')
            if os.path.exists(image_dir):
                for filename in os.listdir(image_dir):
                    src_file = os.path.join(image_dir, filename)
                    dst_file = os.path.join(class_dir, filename)

                    # Move the file
                    shutil.move(src_file, dst_file)

                print(f"  - Moved images to {class_name}/")

                # 3. Remove the now empty 'image' folder
                os.rmdir(image_dir)
                print(f"  - Removed image folder")

    print("\nRestructuring complete.")

In [5]:
print_dataset_folder_structure(KAGGLE_DATASET_PATH)

EBHI-SEG/
    Adenocarcinoma/
        label/
        image/
    Normal/
        label/
        image/
    Low-grade IN/
        label/
        image/
    Serrated adenoma/
        label/
        image/
    High-grade IN/
        label/
        image/
    Polyp/
        label/
        image/


In [6]:
restructure_dataset(KAGGLE_DATASET_PATH)

Processing: Adenocarcinoma
  - Removed label folder
  - Moved images to Adenocarcinoma/
  - Removed image folder
Processing: Normal
  - Removed label folder
  - Moved images to Normal/
  - Removed image folder
Processing: Low-grade IN
  - Removed label folder
  - Moved images to Low-grade IN/
  - Removed image folder
Processing: Serrated adenoma
  - Removed label folder
  - Moved images to Serrated adenoma/
  - Removed image folder
Processing: High-grade IN
  - Removed label folder
  - Moved images to High-grade IN/
  - Removed image folder
Processing: Polyp
  - Removed label folder
  - Moved images to Polyp/
  - Removed image folder

Restructuring complete.


In [7]:
print_dataset_folder_structure(KAGGLE_DATASET_PATH)

EBHI-SEG/
    Adenocarcinoma/
    Normal/
    Low-grade IN/
    Serrated adenoma/
    High-grade IN/
    Polyp/


In [9]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

dataset = ImageFolder(KAGGLE_DATASET_PATH, transform=transforms.ToTensor())